In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/balance-fer2013-data-image/__results__.html
/kaggle/input/balance-fer2013-data-image/balanced_emotions_dataset.csv
/kaggle/input/balance-fer2013-data-image/__notebook__.ipynb
/kaggle/input/balance-fer2013-data-image/__output__.json
/kaggle/input/balance-fer2013-data-image/custom.css
/kaggle/input/balance-fer2013-data-image/__results___files/__results___8_0.png
/kaggle/input/balance-fer2013-data-image/__results___files/__results___11_0.png
/kaggle/input/balance-fer2013-data-image/__results___files/__results___22_0.png


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppresses all warnings (only errors will be shown)

In [3]:
import numpy as np
import pandas as pd
import cv2  # For image processing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

E0000 00:00:1731088262.349440      77 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:479
D1108 17:51:02.357546800      77 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D1108 17:51:02.357563993      77 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D1108 17:51:02.357567660      77 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D1108 17:51:02.357570382      77 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D1108 17:51:02.357573211      77 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defau

# Load Data

In [4]:
data = pd.read_csv("/kaggle/input/balance-fer2013-data-image/balanced_emotions_dataset.csv")
data.head()

,emotion,pixels,Usage,emotion_text
0,0,143 144 145 145 146 145 151 149 142 135 128 13...,Training,anger
1,0,182 167 154 146 134 84 30 10 3 17 27 10 7 11 6...,Training,anger
2,0,207 208 208 209 208 214 176 139 116 66 124 135...,Training,anger
3,0,193 193 196 203 216 229 233 243 189 114 87 95 ...,Training,anger
4,0,168 195 207 208 209 207 208 204 207 207 208 20...,Training,anger


In [5]:
data.shape

(28000, 4)

In [6]:
print(f"Length of pixel data: {len(data.pixels[0].split())}")

Length of pixel data: 2304


# Preproccess in chunks
because of the limited ressources

In [7]:
def preprocess_pixels(img_array):
    # Ensure the input is already a NumPy array, so no need to split or convert again
    img_array = img_array.reshape(48, 48)  # Reshape to 48x48
    img_array = np.expand_dims(img_array, -1)  # Add channel dimension for grayscale
    img_array = np.repeat(img_array, 3, axis=-1)  # Convert to 3 channels for Xception
    img_array = cv2.resize(img_array, (299, 299)) / 255.0  # Resize to 299x299 and normalize
    return img_array


In [8]:
# Process data in chunks
def preprocess_chunk(chunk):
    chunk['pixels'] = chunk['pixels'].apply(lambda x: np.array(x.split(), dtype='float32'))
    chunk['image'] = chunk['pixels'].apply(preprocess_pixels)  # Pass the array directly to preprocess_pixels
    return chunk

In [9]:
# Load data in chunks
chunk_size = 1000
chunks = []
for chunk in pd.read_csv("/kaggle/input/balance-fer2013-data-image/balanced_emotions_dataset.csv",chunksize=chunk_size):
    chunks.append(chunk)

In [10]:
# Apply preprocessing to each chunk
processed_chunks = []
for chunk in chunks:
    processed_chunk = preprocess_chunk(chunk)
    processed_chunks.append(processed_chunk)
      
final_data = pd.concat(processed_chunks, ignore_index=True)

In [11]:
final_data

,emotion,pixels,Usage,emotion_text,image
0,0,"[143.0, 144.0, 145.0, 145.0, 146.0, 145.0, 151...",Training,anger,"[[[0.56078434, 0.56078434, 0.56078434], [0.560..."
1,0,"[182.0, 167.0, 154.0, 146.0, 134.0, 84.0, 30.0...",Training,anger,"[[[0.7137255, 0.7137255, 0.7137255], [0.713725..."
2,0,"[207.0, 208.0, 208.0, 209.0, 208.0, 214.0, 176...",Training,anger,"[[[0.8117647, 0.8117647, 0.8117647], [0.811764..."
3,0,"[193.0, 193.0, 196.0, 203.0, 216.0, 229.0, 233...",Training,anger,"[[[0.75686276, 0.75686276, 0.75686276], [0.756..."
4,0,"[168.0, 195.0, 207.0, 208.0, 209.0, 207.0, 208...",Training,anger,"[[[0.65882355, 0.65882355, 0.65882355], [0.658..."
...,...,...,...,...,...
27995,1,"[48.0, 34.0, 21.0, 18.0, 16.0, 21.0, 26.0, 36....",PrivateTest,disgust,"[[[0.1882353, 0.1882353, 0.1882353], [0.188235..."
27996,1,"[98.0, 103.0, 107.0, 105.0, 100.0, 103.0, 108....",PrivateTest,disgust,"[[[0.38431373, 0.38431373, 0.38431373], [0.384..."
27997,1,"[247.0, 247.0, 247.0, 246.0, 252.0, 224.0, 150...",PrivateTest,disgust,"[[[0.96862745, 0.96862745, 0.96862745], [0.968..."
27998,1,"[186.0, 146.0, 50.0, 42.0, 43.0, 35.0, 48.0, 9...",PrivateTest,disgust,"[[[0.7294118, 0.7294118, 0.7294118], [0.729411..."


# Verify the preproccesed data

In [12]:
final_data['image'][0].shape

(299, 299, 3)

In [13]:
final_data['image'][0].min()

0.073392905

In [14]:
final_data['image'][0].max()

1.0

# Drop useless columns before saving

In [15]:
final_data = final_data.drop(columns=['pixels', 'Usage'])

In [16]:
final_data.head()

,emotion,emotion_text,image
0,0,anger,"[[[0.56078434, 0.56078434, 0.56078434], [0.560..."
1,0,anger,"[[[0.7137255, 0.7137255, 0.7137255], [0.713725..."
2,0,anger,"[[[0.8117647, 0.8117647, 0.8117647], [0.811764..."
3,0,anger,"[[[0.75686276, 0.75686276, 0.75686276], [0.756..."
4,0,anger,"[[[0.65882355, 0.65882355, 0.65882355], [0.658..."


In [17]:
!pip install tables

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/7.6 MB ? eta -:--:--

     ━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.2/7.6 MB 7.0 MB/s eta 0:00:02

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━ 5.2/7.6 MB 56.4 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━ 5.2/7.6 MB 56.4 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 7.6/7.6 MB 44.2 MB/s eta 0:00:01

...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.0/405.0 kB 27.6 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/4.4 MB ? eta -:--:--

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 74.9 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.5/475.5 kB 36.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


# Use .npz to save 3D data and .h5 for the labels

In [18]:
np.savez_compressed('images_data.npz', images=final_data['image'].values)

In [19]:
# Remove the 'image' column before saving the DataFrame
final_data_no_image = final_data.drop(columns=['image'])

# Now save the rest of the data (emotion, usage, etc.)
final_data_no_image.to_hdf('structured_data.h5', key='df', mode='w')

# Load and Verify the shape

In [20]:
import numpy as np

# Load the images from the compressed .npz file
data = np.load('/kaggle/working/images_data.npz',allow_pickle=True)
images = data['images']


In [21]:
images[0].shape

(299, 299, 3)

In [22]:
structured_data = pd.read_hdf('structured_data.h5', 'df')
labels = structured_data['emotion'].values

In [23]:
from sklearn.model_selection import train_test_split

# Split data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)

# Check the split
print("Training data shape:", X_train.shape)
print("Validation data shape:", X_val.shape)

Training data shape: (22400,)
Validation data shape: (5600,)
